In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv("../input/health-insurance-cross-sell-prediction/train.csv")
dataset.head()

### checking for missing data

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.info()

In [ ]:
dataset['Gender'] = dataset['Gender'].replace(to_replace = ['Male','Female'],value = [0,1])
dataset['Vehicle_Damage'] = dataset['Vehicle_Damage'].replace(to_replace = ['Yes','No'], value = [1,0])

In [ ]:
dataset.head()

In [ ]:
dataset['Vehicle_Age'].value_counts()

### Exploratory data analysis

In [ ]:
sns.barplot('Response', 'Annual_Premium', data = dataset)

In [ ]:
sns.barplot("Vehicle_Age", "Response", data = dataset)

#### vehicle more than 2 years old has more insurance rate

In [ ]:
sns.barplot("Gender", "Response", data = dataset)

In [ ]:
men_in = [1 if dataset["Gender"][i] == 0 and dataset['Response'][i] == 1 else 0 for i in range(len(dataset["Gender"]))]
men_out = [1 if dataset["Gender"][i] == 0 and dataset['Response'][i] == 0 else 0 for i in range(len(dataset["Gender"]))]
women_out = [1 if dataset["Gender"][i] == 1 and dataset['Response'][i] == 0 else 0 for i in range(len(dataset["Gender"]))]
women_in = [1 if dataset["Gender"][i] == 1 and dataset['Response'][i] == 1 else 0 for i in range(len(dataset["Gender"]))]

In [ ]:
print("total customers: ", len(dataset["Gender"]))
print("men who insuranced: ",sum(men_in))
print("men who did not buy insurance:", sum(men_out))
print("women who insuranced: ",sum(women_in))
print("women who did not buy insurance:", sum(women_out))

#### more men insuranced than women (men insured 56% more than women)

In [ ]:
sns.barplot("Driving_License", "Response", data = dataset)

#### people with Driving license insuranced more

In [ ]:
sns.barplot("Previously_Insured", "Response", data = dataset)

#### people who previously insuranced DID NOT insurance this time, maybe they already have a policy or this company is just bad

In [ ]:
sns.FacetGrid(data= dataset, hue="Response", height=5).map(sns.distplot, "Age").add_legend()

#### Younger people did not insurance. while the avg insurance age is around 40-50

In [ ]:
plt.figure(figsize = (16,16))
sns.heatmap(dataset.corr(),annot = True, cmap = "Blues")
plt.show()

In [ ]:
dataset['Vehicle_Age'] = dataset['Vehicle_Age'].replace(to_replace = ["1-2 Year","< 1 Year","> 2 Years"], value =[1,0,2] )

In [ ]:
dataset_new = dataset.drop(['Region_Code', 'Policy_Sales_Channel','Vintage','Annual_Premium'], axis = 1)
dataset_new.head()

In [ ]:
x = dataset_new.iloc[:,:-1].values
y = dataset_new.iloc[:,-1].values
print(pd.DataFrame(x)) #coz i am retarded

### choosing the models

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# train_test split is used only for accuracy measurement before submission, has nothing to do with competition
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2,random_state = 177013)

In [ ]:
XGB = XGBClassifier(n_estimators = 1000, alpha = 0.1, random_state = 177013)
LR = LogisticRegression()
DTC = DecisionTreeClassifier()

In [ ]:
XGB.fit(x_train,y_train)
LR.fit(x_train,y_train)
DTC.fit(x_train,y_train)

In [ ]:
y_pred_XGB = XGB.predict(x_test)
y_pred_LR = LR.predict(x_test)
y_pred_DTC = DTC.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
cm1 = accuracy_score(y_pred_XGB,y_test)
cm2 = accuracy_score(y_pred_LR,y_test)
cm3 = accuracy_score(y_pred_DTC,y_test)
print("XGB score: {0} \nLR score : {1} \nDTC score : {2}".format(cm1,cm2,cm3))

#### since Logistic Regression has more accuracy. ~i will use it for submission.~ LR seems to not classify anything as 1,
#### so, i will go with XGboost 

### test_set

In [ ]:
def preprocessing(dataset):
    dataset['Gender'] = dataset['Gender'].replace(to_replace = ['Male','Female'],value = [0,1])
    dataset['Vehicle_Damage'] = dataset['Vehicle_Damage'].replace(to_replace = ['Yes','No'], value = [1,0])
    dataset['Vehicle_Age'] = dataset['Vehicle_Age'].replace(to_replace = ["1-2 Year","< 1 Year","> 2 Years"], value =[1,0,2] )
    dataset = dataset.drop(['Region_Code', 'Policy_Sales_Channel','Vintage','Annual_Premium'], axis = 1)
    return dataset

In [ ]:
test_set = pd.read_csv("../input/health-insurance-cross-sell-prediction/test.csv")
testset = preprocessing(test_set)
testset

In [ ]:
test = testset.iloc[:,:].values 
test

In [ ]:
y_pred = DTC.predict(test)

In [ ]:
submission = pd.DataFrame(y_pred,testset['id'])

In [ ]:
submission = submission.rename(columns = {0:'Response'})

In [ ]:
submission.to_csv('./submission.csv')